In [269]:
cmd

'curl \'https://www.zvg-portal.de/index.php?button=Suchen\'   -H \'Connection: keep-alive\'   -H \'Cache-Control: max-age=0\'   -H \'sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"\'   -H \'sec-ch-ua-mobile: ?0\'   -H \'Upgrade-Insecure-Requests: 1\'   -H \'Origin: https://www.zvg-portal.de\'   -H \'Content-Type: application/x-www-form-urlencoded\'   -H \'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36\'   -H \'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\'   -H \'Sec-Fetch-Site: same-origin\'   -H \'Sec-Fetch-Mode: navigate\'   -H \'Sec-Fetch-User: ?1\'   -H \'Sec-Fetch-Dest: document\'   -H \'Referer: https://www.zvg-portal.de/index.php?button=Termine%20suchen\'   -H \'Accept-Language: en-GB,en;q=0.9\'   --data-raw \'ger_name=--+Alle+Amtsgerichte+--&order_by=2&land_abk=bw&ger_id=0&

In [387]:
def get_curl_cmd_search_pages(land_abk = "bw", page_no = "1"):
    string1 = "curl 'https://www.zvg-portal.de/index.php?button=Suchen&seite="
    string2 = page_no + """&l=1&r=8&all=&order_by=-1&desc=-1'   -H 'Connection: keep-alive'   -H 'Cache-Control: max-age=0'   -H 'sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"'   -H 'sec-ch-ua-mobile: ?0'   -H 'Upgrade-Insecure-Requests: 1'   -H 'Origin: https://www.zvg-portal.de'   -H 'Content-Type: application/x-www-form-urlencoded'   -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'   -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'   -H 'Sec-Fetch-Site: same-origin'   -H 'Sec-Fetch-Mode: navigate'   -H 'Sec-Fetch-User: ?1'   -H 'Sec-Fetch-Dest: document'   -H 'Referer: https://www.zvg-portal.de/index.php?button=Termine%20suchen'   -H 'Accept-Language: en-GB,en;q=0.9'   --data-raw 'ger_name=--+Alle+Amtsgerichte+--&order_by=2&land_abk="""
    string3 = land_abk + """&ger_id=0&az1=&az2=&az3=&az4=&art=&obj=&str=&hnr=&plz=&ort=&ortsteil=&vtermin=&btermin='"""
    string = string1 + string2 + string3
    return string

def get_curl_cmd_result_detail(url = ""):
    cmd = """curl '{url}' \
          -H 'Connection: keep-alive' \
          -H 'Cache-Control: max-age=0' \
          -H 'sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"' \
          -H 'sec-ch-ua-mobile: ?0' \
          -H 'Upgrade-Insecure-Requests: 1' \
          -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36' \
          -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
          -H 'Sec-Fetch-Site: same-origin' \
          -H 'Sec-Fetch-Mode: navigate' \
          -H 'Sec-Fetch-User: ?1' \
          -H 'Sec-Fetch-Dest: document' \
          -H 'Referer: https://www.zvg-portal.de/index.php?button=Suchen' \
          -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8,de;q=0.7' \
  --compressed""".format(url = url)
    return cmd

In [367]:
string = get_curl_cmd_search_pages()

In [410]:
import shlex
import subprocess
import chardet

def execute_command(cmd):
    args = shlex.split(cmd)
    process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    #print(chardet.detect(stdout)['encoding'])
    result = stdout.decode("utf-8","ignore") 
    return result, stdout, stderr

In [388]:
HTML(execute_command(get_curl_cmd_result_detail("https://www.zvg-portal.de/index.php?button=showZvg&zvg_id=7352&land_abk=bw"))[0])

ISO-8859-1


In [851]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [1203]:
import datetime
def replace_all(text, dic):
    found = False
    for i, j in dic.items():
        if i in text:
            text = j
            found = True
    if not found:
        print(text)
        text = None
    return text

def scrape_page(land_abk = "bw", page_no = 1):
    page_no = str(page_no)
    string = get_curl_cmd_search_pages(land_abk, page_no)
    soup = BeautifulSoup(execute_command(string)[0])
    pages_buttons = soup.find_all("button", {"class": "seiten_nr"})
    n_pages = len(pages_buttons)//2
    
    objekt_lage = [td.parent for td in soup.findAll("td", string = "Objekt/Lage")]
    amtsgericht = [ol.find_previous_sibling('tr') for ol in objekt_lage]
    aktenzeichen_link = [am.find_previous_sibling('tr') for am in amtsgericht]
    verkehrswert = [ol.find_next_sibling('tr') for ol in objekt_lage]
    vks_text = [str(list(v.children)[1].text) for v in verkehrswert]
    termin = [val.find_next_sibling('tr') for val in verkehrswert]
    
    links = ["https://www.zvg-portal.de/"+a.findAll("a")[0]["href"] for a in aktenzeichen_link]
    aktenzeichen = [re.search("<nobr>(.*/.*[0-9]{4}).*</nobr>",str(link.findAll("nobr")[1])).groups(1)[0] for link in aktenzeichen_link]
    plz = [re.search("[0-9]{5}", str(a)).group() for a in objekt_lage]
    address = [re.search("</b> (.*[0-9]{5}.*)</td>", str(a)).groups(1)[0] for a in objekt_lage]
    
    days = [re.search("([0-9]+)\..*",t.text).groups(1)[0] for t in termin]
    months = [re.search("[0-9]+\. ([a-zA-Z]+) [0-9]+.*",t.text).groups(1)[0] for t in termin]
    years = [re.search("[0-9]+\. [a-zA-Z]+ ([0-9]{4}).*",t.text).groups(1)[0] for t in termin]

    months_to_numbers = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "Mai":"5","Jun":"6","Jul":"7","Aug":"8","Sep":"9","Okt":"10","Nov":"11","Dez":"12"}
    months1 = [replace_all(m, months_to_numbers) for m in months]
    
    dates = []
    for y, m, d in zip(years, months1, days):
        try:
            dates.append(pd.Timestamp(int(y),int(m),int(d)))
        except:
            print(y)
            print(m)
            print(d)
            
    amtsgericht = [list(a.children)[1].text for a in amtsgericht]
    objekt_lage = [list(a.children)[1].text for a in objekt_lage]
 
    vals = [list(re.findall("[0-9]*\.*[0-9]*\.*[0-9]*\.*[0-9]*\.*[0-9]+,[0-9\-]+",str(val.text))) for val in verkehrswert]
    vals = [str([item.replace("-","0").replace(".",";").replace(",",".").replace(";",",") for item in sublist]) for sublist in vals]
    
    data = [aktenzeichen, amtsgericht, links, plz, address, objekt_lage, vals, vks_text, dates]
    
    df = pd.DataFrame(data).T
    df.columns = ["Aktenzeichen", "Amtsgericht", "Links", "PLZ", "Address", "Objekt/Lage", "Verkehrswerte", "Verkehrswert Text",  "Termin"]
    return n_pages, soup, df

In [1210]:
scrape_page()[2]["Termin"]

0   2021-05-25
1   2021-06-08
2   2021-06-10
3   2021-06-10
4   2021-06-10
5   2021-06-10
6   2021-06-14
7   2021-06-16
8   2021-06-17
Name: Termin, dtype: datetime64[ns]

In [1204]:
df['Termin'] = df['Termin'].apply(lambda x: pd.Timestamp(x))

In [1205]:
n_pages, soup, data = scrape_page()

In [1206]:
pd.set_option('display.max_colwidth', None)

In [1207]:
pd.Timestamp(df["Termin"][0])

Timestamp('2021-05-25 00:00:00')

https://www.zvg-portal.de/index.php?button=showZvg&zvg_id=7361&land_abk=bw


https://www.zvg-portal.de/index.php?button=showZvg&zvg_id=7361&land_abk=bw

In [1024]:
n_pages

7

In [1025]:
address

['Eschstraße 14, 88074 Meckenbeuren',
 'Unterzeil , 88299  Leutkirch , Reichenhofen ',
 'Beckenkreuz  11, 88471 Laupheim, Untersulmetingen',
 'Hubenstr. 25, 68305 Mannheim, Waldhof',
 'Mozartstraße 16, 77746 Schutterwald',
 'Berliner Str. 7, 69502 Hemsbach',
 'Jakob-Faulhaber-Str. 2, Schienenstr. 11, 68305 Mannheim, Waldhof']

In [1072]:
def scrape_pages(land_abk = "be"):
    n_pages, _, data = scrape_page(land_abk, 1)
    print(".", end = "")
    if n_pages > 1:
        for i in range(2, n_pages+1):
            n_pages, _, data1 = scrape_page(land_abk, i)
            data = data.append(data1, ignore_index = True)
            print(".",end= "")
    return data

In [1181]:
def scrape_all(verbose = 0):
    land_abks = ["bw", "by", "be", "br", "hb", "hh", "he", "mv", "no", "nw", "rp", "sl", "sn", "st", "sh", "th"]
    dfs = scrape_pages(land_abks[0])
    for abk in land_abks[1:]:
        dfs = dfs.append(scrape_pages(abk), ignore_index=True)
        if verbose == 0:
            print("-", end="")
    return dfs

In [1303]:
with open("dfs.pkl", "rb") as f:
    dfs = pickle.load(f)

In [1305]:
dfs

AttributeError: 'DataFrame' object has no attribute '_data'

In [1295]:
with open("plz_to_long_lat.json", "r") as f:
    plz_to_long_lat = json.loads(f.read())
long, lat = [], []
for line in dfs.items():
    long.append(plz_to_long_lat(line["PLZ"])[0])
    lat.append(plz_to_long_lat(line["PLZ"])[1])
dfs.insert(len(dfs.keys()), "long", long)
dfs.insert(len(dfs.keys()), "lat", lat)

AttributeError: 'DataFrame' object has no attribute '_data'

In [1128]:
import ast

In [1129]:
def unpack_market_values(dfs):
    return [[float(val.replace(',','')) for val in ast.literal_eval(item)] for item in list(dfs["Verkehrswerte"].array)]

# Why missing Verkehrswerte?

In [1139]:
idxs = np.argwhere(np.array([len(x) for x in unpack_market_values(dfs)]) == 0 )
for i in idxs:
    print(dfs[["Verkehrswert Text", "Verkehrswerte"]].iloc[i])

   Verkehrswert Text Verkehrswerte
72       siehe oben!            []
   Verkehrswert Text Verkehrswerte
89       siehe oben             []
    Verkehrswert Text Verkehrswerte
113       siehe oben             []
    Verkehrswert Text Verkehrswerte
114       siehe oben             []
    Verkehrswert Text Verkehrswerte
117       siehe oben             []
    Verkehrswert Text Verkehrswerte
118       siehe oben!            []
    Verkehrswert Text Verkehrswerte
157      283.000.00 €            []
    Verkehrswert Text Verkehrswerte
158         720.000 €            []
      Verkehrswert Text Verkehrswerte
174  siehe Beschreibung            []
    Verkehrswert Text Verkehrswerte
176              1502            []
       Verkehrswert Text Verkehrswerte
192  Siehe Beschreibung             []
    Verkehrswert Text Verkehrswerte
267         274.000.-            []
    Verkehrswert Text Verkehrswerte
302       37.000 Euro            []
    Verkehrswert Text Verkehrswerte
314       471.000 EUR 

In [1184]:
dfs.columns

Index(['Aktenzeichen', 'Amtsgericht', 'Links', 'PLZ', 'Address', 'Amtsgericht',
       'Objekt/Lage', 'Verkehrswerte', 'Verkehrswert Text', 'Termin'],
      dtype='object')

In [1179]:
from datetime import date
from scraping import *
today = ""
dfs = None
def update_scrape():
    global today
    global dfs
    
    if str(date.today()) != today:
        dfs = scrape_all(verbose =  1)
        today = str(date.today())

In [1180]:
update_scrape()
print(today)

2021-05-21


In [1136]:
print(dfs[["Verkehrswert Text", "Verkehrswerte", "Objekt/Lage"]].iloc[113])

Verkehrswert Text                                                      siehe oben 
Verkehrswerte                                                                   []
Objekt/Lage          Einfamilienhaus mit 2 Garagen: Am Vielitzberg  38, 95100 Selb
Name: 113, dtype: object


In [1123]:
re.findall("[0-9]*\.*[0-9]*\.*[0-9]*\.*[0-9]*\.*[0-9]+,[0-9\-]+","1.1173,00")

['1.1173,00']

In [277]:
HTML(str(soup))

In [149]:
import re

In [148]:
pages_buttons = soup.find_all("button", {"class": "seiten_nr"})
n_pages = len(pages_buttons)//2

In [153]:
auctions = [td.parent for td in soup.findAll("td", string="Objekt/Lage")]

In [154]:
auctions

[<tr><td align="left" valign="top">Objekt/Lage</td><td align="left" colspan="2" valign="center"><b>Eigentumswohnung (3 bis 4 Zimmer), Garage<!--Lage--->:</b> Eschstraße 14, 88074 Meckenbeuren</td></tr>,
 <tr><td align="left" valign="top">Objekt/Lage</td><td align="left" colspan="2" valign="center"><b>unbebautes Grundstück<!--Lage--->:</b> Unterzeil , 88299  Leutkirch , Reichenhofen </td></tr>,
 <tr><td align="left" valign="top">Objekt/Lage</td><td align="left" colspan="2" valign="center"><b>Wohnung und Tiefgaragenstellplatz<!--Lage--->:</b> Beckenkreuz  11, 88471 Laupheim, Untersulmetingen</td></tr>,
 <tr><td align="left" valign="top">Objekt/Lage</td><td align="left" colspan="2" valign="center"><b>Eigentumswohnung (3 bis 4 Zimmer)<!--Lage--->:</b> Hubenstr. 25, 68305 Mannheim, Waldhof</td></tr>,
 <tr><td align="left" valign="top">Objekt/Lage</td><td align="left" colspan="2" valign="center"><b>Einfamilienhaus, Garage<!--Lage--->:</b> Mozartstraße 16, 77746 Schutterwald</td></tr>,
 <tr><

In [233]:
plz = []
address = []
for a in auctions:
    try:
        match = re.search("[0-9]{5}", str(a))
        plz.append(match.group())
        match = re.search("</b> (.*[0-9]{5}.*)</td>", str(a))
        address.append(match.group(1))
    except Exception as e:
        print(e)

In [234]:
address

['Eschstraße 14, 88074 Meckenbeuren',
 'Unterzeil , 88299  Leutkirch , Reichenhofen ',
 'Beckenkreuz  11, 88471 Laupheim, Untersulmetingen',
 'Hubenstr. 25, 68305 Mannheim, Waldhof',
 'Mozartstraße 16, 77746 Schutterwald',
 'Berliner Str. 7, 69502 Hemsbach',
 'Jakob-Faulhaber-Str. 2, Schienenstr. 11, 68305 Mannheim, Waldhof']

In [1243]:
with open("plz_to_long_lat.json", "r") as f:
    jsons = f.read()

In [1267]:
for x in dfs.items():
    print(x)
    pass

('Aktenzeichen', 0      0001 K 0020/2019
1      0002 K 0019/2018
2      0003 K 0095/2020
3      0001 K 0001/2020
4      0001 K 0071/2020
             ...       
534    0063 K 0021/2020
535    0061 K 0019/2019
536    0061 K 0036/2019
537    0061 K 0040/2019
538    0061 K 0012/2019
Name: Aktenzeichen, Length: 539, dtype: object)
('Amtsgericht', 0       Ravensburg in Baden-Wuerttemberg
1         Biberach in Baden-Wuerttemberg
2         Mannheim in Baden-Wuerttemberg
3        Offenburg in Baden-Wuerttemberg
4         Mannheim in Baden-Wuerttemberg
                     ...                
534       Elmshorn in Schleswig-Holstein
535    Norderstedt in Schleswig-Holstein
536    Norderstedt in Schleswig-Holstein
537    Norderstedt in Schleswig-Holstein
538    Norderstedt in Schleswig-Holstein
Name: Amtsgericht, Length: 539, dtype: object)
('Links', 0      https://www.zvg-portal.de/index.php?button=showZvg&zvg_id=7352&land_abk=bw
1      https://www.zvg-portal.de/index.php?button=showZvg&zvg_id=

In [1229]:
list_of_dicts = json.loads(jsons)

In [1236]:
plz_to_long_lat = {}
for d in list_of_dicts:
    plz_to_long_lat[d["zipcode"]] = [d["longitude"], d["latitude"]]

In [1242]:
with open("plz_to_long_lat.json", "w") as f:
    f.write(json.dumps(plz_to_long_lat))